# Project: Data Science - Surgical Duration

Group 31

Sara-Jane Bittner


Xiao-Lan Bokma

http://epydoc.sourceforge.net/epytext.html

# Chapter 1 - Setup

In [1]:
#imports
import os #get the os filepath
import pandas as pd
from collections import Counter

import statsmodels.api as sm  
from scipy.stats import zscore

from matplotlib.pyplot import boxplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

from numpy import mean, std, nan, float64, abs, delete, absolute, sqrt, array

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder,LabelBinarizer
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import f1_score, recall_score, average_precision_score, balanced_accuracy_score
from sklearn import model_selection, svm, tree
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from nltk.probability import FreqDist
from operator import itemgetter
from pprint import pprint

In [2]:
#This will allow our project to allways be executed in the same manor, providing us with a stable result.
randomSeed=12345

# Chapter 2 - Used functions
The following will provide us with all the functions, that are used in our pipeline in chapter 3



### 2.1 Import the data

In [3]:
def import_df_from_file(filename: str):
    """
    Get the path of this file and look into the data folder that is located in the same folder as this file,
    read the .csv file and delete the last row, as it is empty

    @type filename: str
    @param filename: The filename of the .csv spreadsheet
    @rtype: dataframe
    @returns: dataFrame appearing in the .csv file
    """
    
    path = os.path.join('data', filename)
    dataFrame=pd.read_csv(path,sep=';',encoding="ISO-8859-1")
    dataFrame.drop(dataFrame.tail(1).index,inplace=True)
    display(dataFrame)
    return dataFrame

## 2.2 - Preprocessing

### 2.2.1 Retype the columns
Currently, all the dataframe columns are of type "Object", since the data is in string format. All the columns that are not in this shape, will need to be retyped.

The columns that should be float64 are predefined already. The columns that are not recognized as float64 yet will undergo the modification. To retype the float64 columns, the string 'Onbekend' needs to be replaced with np.nan values. 

In [4]:
def replace_with_nan(dataFrame, value):
    """
    Replace specific a value in this dataframe to a np.nan value.
    
    @type dataFrame: dataframe
    @param dataFrame: The dataframe with value that needs replacement.
    @type value: string
    @param value: The value that needs to be replaced by np.nan values
    @rtype: dataframe
    @returns: dataFrame with the values replaced by np.nan values.
    """
    #loop through all the columns in the data and replace the value within each column.
    for column in dataFrame:
        dataFrame[column] = dataFrame[column].replace([value], nan)
    return dataFrame

def get_non_dtype_columns(dataFrame, dTypeColumnNames, dType):
    """
    Get the columns of the dType list that are not yet of that dtype (yet): 
    
    @type dataFrame: dataframe
    @param dataFrame: The dataframe to get the non_dTypeColumnNames columns from.
    @type dTypeColumnNames: list
    @param dTypeColumnNames: list of the column names that should be of the specified dType.
    @type dType: string
    @param dType: the specific dType that the columns should be
    
    @rtype: list
    @returns: non_dTypeColumns which are not of the specific dtype (yet).
    """
    non_dTypeColumnNames = dataFrame[dTypeColumnNames].select_dtypes(exclude=[dType]).columns
    return non_dTypeColumnNames

def convert_df_type(dataFrame, columns, dType):
    """
    Convert the dtype of specific columns in the dataFrame to the specified dType.
    
    @type dataFrame: dataframe
    @param dataFrame: The dataframe with columns that need to be converted to the dType.
    @type columns: list
    @param columns: list of the columns that should be the specified dType. 
    @type dType: string
    @param dType: the dtype that the columns should be    
    @rtype: dataframe
    @returns: dataFrame with the columns 'columns' to dtype 'dType'
    """
    
    if (dType == 'float64'):
        
        print(columns)
        non_dTypeColumnNames = get_non_dtype_columns(dataFrame, columns, dType)
        
        #change the columns that should be float64 but aren't yet.
        # if the float value cannot be determined due to the decimal 
        # separator, commas will be resplaced by periods.
        for columnName in non_dTypeColumnNames:
            if dataFrame[columnName].dtype=="int8" or dataFrame[columnName].dtype=="int64":
                dataFrame[columnName] = dataFrame[columnName].astype(float)
            else:
                dataFrame[columnName] = dataFrame[columnName].str.replace(',', '.').astype(float)
            
    #retype all the object columns to category type
    elif (dType == 'category'):
        dataFrame = pd.concat([
        dataFrame.select_dtypes([], ['object']),
        dataFrame.select_dtypes(['object']).apply(pd.Series.astype, dtype=dType)
        ], axis=1)
    
    #convert the columns that are categories into intergers, 
    # based on the provided list of columns
    elif (dType == 'int8'):
        for columnName in columns:
            dataFrame[columnName] = dataFrame[columnName].astype('category').cat.codes
    return dataFrame

### 2.2.2 Remove the Rows and Columns with too much NAN

Drop columns that have more than 50% nan values. Drop the rows that will have one or more missing cells.

In [5]:
def dropWithTooMuchNan(dataFrame, axis):
    """
    Drop the row/column that 
    
    @type dataFrame: dataframe
    @param dataFrame: The dataframe that should have Nan rows/columns removed. 
    @param axis: integer
    @param axis: 0 is about the row axis, 1 is the column axis.
    @rtype: dataframe
    @returns: dataFrame with the columns 'columns' to dtype 'dType'
    """
    
    if axis == COLUMN_AXIS:
        allColumnNames = dataFrame.columns
        NANColumns =[]
        for columnName in allColumnNames:
            if dataFrame[columnName].isnull().values.any():
                    howManyNaN = dataFrame[columnName].isnull().sum()
                    if howManyNaN > ((len(dataFrame[columnName])/100)*50):
                        NANColumns += [columnName]
                        dataFrame.drop(labels=columnName,axis=COLUMN_AXIS, inplace= True)
        return dataFrame, NANColumns
    
    elif axis == ROW_AXIS:
        # drop the rows that have missing values.
        dataFrame.dropna(thresh= len(dataFrame.columns), inplace = True)
        dataFrame.isnull().values.any()
        dataFrame = dataFrame.reset_index(drop=True)
    
    return dataFrame

### 2.2.3 Creation of new columns

Creating the Overtime column based on the current planned Duration time.

In [6]:
def createOvertimeColumn(dataFrame):
    """
    Create the Overtime column by substracting ActualDurationTime from the PlannedDurationTime
    
    @type dataFrame: dataframe
    @param dataFrame: The dataframe that containt the two columns that should become a new column.
    @rtype: dataframe
    @returns: dataFrame with the extra "Overtime" column.
    """
    dataFrame['Overtime'] = dataFrame['ActualDurationTime'] - dataFrame['PlannedDurationTime']
    RANGE_COLUMN_NAMES.append('Overtime')
    ORDINAL_COLUMN_NAMES.append('Overtime')
    return dataFrame

Create the OperationType Columns

In [7]:
class OPTypeEncoder:
    relevant_op_types=["other"] #list of the n% most common operations
    relevant_op_types_Names=["OPType_other"]
    n=99
    def __init__(self, n):
        self.n=n
    
    def splitOPTypeText(self,text):
        thisPatientsOPs=str(text).lower().strip().split(" + ")
        return [i.strip() for i in thisPatientsOPs]
        
    def fit(self,dataframe, columnname):
        """creates a list of relevant operationtypes."""
        
        #get a list of all operations, that have ever been done
        completeOPList=[]
        for patient in dataframe[columnname]:
            #print(patient)
            thisPatientsOPs=self.splitOPTypeText(patient)
            completeOPList+=thisPatientsOPs
        
        #find out how often they have been done
        to_n_uses=FreqDist(completeOPList)
        dictionary_items = to_n_uses.items()
        to_n_uses = sorted(dictionary_items, key=itemgetter(1), reverse=True )
        
        #get the operations, that together are used in n% percent of the cases
        threshold=(self.n*len(dataframe[columnname]))/100 # Define how many cases should be covered by the amount of operations.
        i=0
        for ot in to_n_uses:
            #print(ot)
            if ot[0] == "nan":
                pass
            else:
                i+=ot[1]
                self.relevant_op_types.append((ot[0]))
                if i>=threshold:
                    break
        print("The relevant op types, that cover at least",str(self.n)+"% of all operations, are:", self.relevant_op_types)
        self.relevant_op_types_Names=["OPType_"+i for i in self.relevant_op_types]
        
        
    def transform(self,dataframe, columnname):
        
        allPatientsOHE=[]
        
        #add J if this operation was done for this patient, 
        #add N if this operation was not done for this patient
        #add nan, if the operation was unknown
        for patient in dataframe[columnname]:
            #print(patient)
            thisPatientsOPs=self.splitOPTypeText(patient)
            
            #add a column for each relevant operationtype
            thisPatientsOHE=["N"]*len(self.relevant_op_types)
            for op in thisPatientsOPs:
                if op =="nan":
                    thisPatientsOHE=[nan]*len(self.relevant_op_types)
                elif op in self.relevant_op_types:
                    i=self.relevant_op_types.index(op)
                    thisPatientsOHE[i]="J"
                elif not(op in self.relevant_op_types):
                    thisPatientsOHE[0]="J"
            allPatientsOHE.append(thisPatientsOHE)
         
        #turn the results into a dataframe
        allPatientsOHE_DF=pd.DataFrame(allPatientsOHE, columns= self.relevant_op_types_Names)
        
        #remove old column from df
        dataframe = dataframe.drop(labels = columnname, axis=COLUMN_AXIS)
        
        #add new columns to df
        dataframe = pd.merge(dataframe, allPatientsOHE_DF, left_index=True, right_index=True)
        
        #edit
        CATEGORICAL_COLUMN_NAMES.remove(columnname)
        SURGERY_FEATURES.remove(columnname)
        for column in self.relevant_op_types_Names:
            BINARY_COLUMN_NAMES.append(column)
            SURGERY_FEATURES.append(column)
            
        
        return dataframe
        
    def fit_transform(self,dataframe, columnname):
        self.fit(dataframe, columnname)
        return self.transform(dataframe, columnname)

### 2.3.4 Outliers

#### Categorical Outlier Grouping

In [8]:
def groupOutliers(dataFrame, columnName, treshhold):
    '''
    Group the outlier categories into one group, called 'other'.
    
    @type dataFrame: dataframe
    @param dataFrame: the dataframe with the categorical column 
    @type columnNames: list
    @param columnNames: list of the column names that need to be outlier cut
    @type treshold: float
    @param treshold: percentage that needs to be cut off.
    
    @rtype dataFrame: dataframe
    @returns dataFram: dataframe with the cut off outliers based on the treshold.
    '''
    treshAmount = (len(dataFrame)//100)*treshhold
    threshCounter = 0
    countValues = dataFrame[columnName].value_counts()
    uniqueValues = dataFrame[columnName].unique()
    
    for uniqueValue in reversed(uniqueValues):
        if (threshCounter + countValues[uniqueValue]) <= treshAmount:
            threshCounter += countValues[uniqueValue]
            dataFrame[columnName].replace([uniqueValue],'other', inplace=True)

    dataFrame[columnName] = dataFrame[columnName].cat.remove_unused_categories()
    return dataFrame
    
def categorical_outlier_cutting(dataFrame, columnNames, treshold):
    '''
    Group the outlier categories into one group, called 'other'.
    These outliers will be defined according to the treshold.
    
    @type dataFrame: dataframe
    @param dataFrame: the dataframe with the categorical column 
    @type columnNames: list
    @param columnNames: list of the column names that need to be outlier cut
    @type treshold: float
    @param treshold: percentage that needs to be cut off.
    
    @rtype dataFrame: dataframe
    @returns dataFram: dataframe with the cut off outliers based on the treshold.
    '''
    for column in columnNames:
#         print ("BEFORE: ", dataFrame[column].value_counts())
        dataFrame=groupOutliers(dataFrame, column, treshold)
#         print ("AFTER: ", dataFrame[column].value_counts())
#     print(dataFrame['Surgeon'].value_counts())
    return dataFrame

#### Numerical Outlier Cutting

In [9]:
def numerical_outlier_cutting(dataFrame, numericalColumnNames):
    '''
    Define the numerical outliers, based on the z-scores. Based on the range 2, the outliers will be cut.    
    
    @type dataFrame: dataFrame
    @param dataFrame: the datafram that contains the columns that need to have the values outliercut.
    @type numericalColumnNames: list
    @param numericalColumnNames: list of the columnNames with numerical data.
    
    @rtype dataFrame: dataframe
    @returns dataFrame: dataframe with the numerical outliers cut.
    '''
    
#     minValue = dataFrame[columnName].min()
#     maxValue = dataFrame[columnName].max()

#     print(minValue) 
#     print(maxValue) 

    numericalData = dataFrame[numericalColumnNames]

    z_scores = zscore(numericalData)
    abs_z_scores = abs(z_scores)
    filtered_entries = (abs_z_scores < 2).all(axis=1)
    dataFrame = dataFrame[filtered_entries]
    dataFrame = dataFrame.reset_index(drop=True)

#     minValue = dataFrame['PlannedDurationTime'].min()
#     maxValue = dataFrame['PlannedDurationTime'].max()

#     print(minValue) #minutes
#     print(maxValue) #minutes
    
    return dataFrame

### 2.2.5 Creating categorical data from numerical data ranges.

In [10]:
def get_bins (columnData, columnName):
    minValue = columnData.min()
    maxValue = columnData.max()
    
#     print(minValue) #minutes
#     print(maxValue) #minutes
    
    if columnName == "BMI":    
        bins = [(minValue-1),18.5,24.9,29.9,maxValue]
        binNames = ['underweight', 'normal', 'overweight', 'obese']
    elif columnName == "Age":
        bins = [int(minValue-1), 57, 66,76, int(maxValue)]
        binNames = [ (str(x+1)+'-'+str(y)) for x,y in zip(bins[0::1], bins[1::1]) ]
    elif columnName == "Overtime":
        bins = [(minValue-1),(-70),(-25),25,70, 120,(maxValue)]
        binNames = ['Ahead_Of_Time_Extreme','Ahead_Of_Time_Medium', 'In_Time','Overtime_Small','Overtime_Medium', 'Overtime_Extreme']
    elif columnName == "ActualDurationTime":
        bins = [int(minValue-1), 180, 240, 280, 345,int(maxValue)]
        binNames = [ (str(x+1)+'-'+str(y)) for x,y in zip(bins[0::1], bins[1::1]) ]
    elif columnName == "PlannedDurationTime":
        bins = [int(minValue-1), 180, 240, 280, int(maxValue)]
        binNames = [ (str(x+1)+'-'+str(y)) for x,y in zip(bins[0::1], bins[1::1]) ]    
    return bins, binNames

def convert_categorical_range(dataFrame, columnName):

    bins, binNames = get_bins(dataFrame[columnName], columnName)
    
    dataFrame[columnName + 'Range'] = pd.cut(dataFrame[columnName], bins, labels=binNames)
    #TODO Check <>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
    if columnName is not 'PlannedDurationTime' and columnName is not 'ActualDurationTime': 
        dataFrame.drop(labels = columnName, axis=COLUMN_AXIS, inplace = True)
    
#     replace_columnName_in_lists(columnName, columnName+'Range')
    
    print(dataFrame[columnName + 'Range'].value_counts())
    print (bins)
    print(len(bins))
    print(binNames)
    print(len(binNames))

    # add BMIRange to patientFeatures-List
#     PATIENT_FEATURES += (columnName + 'Range')

    return dataFrame, binNames

def to_categorical_range(dataFrame):
    column2binNames={}
    for columnName in RANGE_COLUMN_NAMES:
        dataFrame, binNames = convert_categorical_range(dataFrame, columnName)
#     dataFrame = convert_categorical_range(dataFrame, 'BMI')
#     dataFrame = convert_categorical_range(dataFrame, 'Age')
# #     dataFrame = convert_categorical_range(dataFrame, 'Overtime')
#     dataFrame = convert_categorical_range(dataFrame, 'ActualDurationTime')
#     dataFrame = convert_categorical_range(dataFrame, 'PlannedDurationTime')
        column2binNames[columnName+"Range"]=binNames
    return dataFrame, column2binNames

<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-10-dbf2687cf347>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if columnName is not 'PlannedDurationTime' and columnName is not 'ActualDurationTime':
<ipython-input-10-dbf2687cf347>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if columnName is not 'PlannedDurationTime' and columnName is not 'ActualDurationTime':


### 2.2.6 Encoding

#### Label Encoding

In [11]:
def label_encoding(dataFrame, columns):
    #print(ORDINAL_COLUMN_NAMES)
    
    columnNameToLE={}

        
    for columnName in columns:
        le = LabelEncoder()
        le = le.fit(dataFrame[columnName])
        dataFrame[columnName] = le.transform(dataFrame[columnName])
        columnNameToLE[columnName]=le
        
    print(dataFrame['ActualDurationTimeRange'].value_counts()) #todo
    return dataFrame, columnNameToLE

#### One Hot Encoding

In [12]:
def IntTransfer(dataFrame, columnName):
    '''
    Make string type floats into int.
    When the decimal separator is ',' replace it with a '.' first.
    
    @type dataFrame: dataframe
    @param dataFrame: 
    @type columnName: string
    @param columnName:
    
    @rtype dataFrame: dataframe
    @returns dataFrame:
    
    '''
    for i in range(0,len(dataFrame[columnName])):
        try:
            dataFrame[columnName][i]= int(float(dataFrame[columnName][i].replace(",",".")))
        except ValueError:
            dataFrame[columnName][i]= dataFrame[columnName][i]
            
    return dataFrame

 
def one_hot_encoding(columnNames, dataFrame):
    '''
    Onehot-encodes the columns of the dataFrame, which are listed in columnNames
    
    @type dataFrame: dataframe
    @param dataFrame:
    
    @rtype dataFrame:
    @returns list:
    @rtype dict: allows for reconecting the name of the column to the corresponding encoder.
    '''   
    
    
    columnNameToOHE={}
    
    #for each column, which needs to be OHEd create an OH-Encoder and the corresponding Encoding. Insert it in the Dataframe
    for columnName in columnNames:
        print(columnName)
        dataFrame = IntTransfer(dataFrame,columnName)
        hotCodedArray=array(dataFrame[columnName][:]).reshape(-1, 1)
        
        #create an OH-Encoder and the corresponding Encoding
        
        #enc= LabelBinarizer()
        enc = OneHotEncoder(handle_unknown='ignore')
        hotCodedArray=enc.fit_transform(hotCodedArray).toarray()
        columnNameToOHE[columnName]=enc
        
        #Find out columnames
        hotCodedcolumns=[]
        for category in list(enc.categories_[0]):
            hotCodedcolumns+=[columnName+"_"+str(category)]
        
        
        #create a new df, using the hot coded columns as Column Names
        hotCoded = pd.DataFrame(hotCodedArray, columns = hotCodedcolumns)

        #remove old data column and add new dataframe
        dataFrame.drop(labels=columnName,axis=COLUMN_AXIS, inplace= True)
        dataFrame = pd.merge(dataFrame, hotCoded, left_index=True, right_index=True)
        
    return dataFrame, columnNameToOHE.keys(), columnNameToOHE

#### Calculate the VIF for One Hot encoding

In [13]:
def calculateVIF(df, featuresToTest):
    
    #Filters the whole dataset for the feature set of one OneHotEncoded-Column
    independentVariables = df.filter(featuresToTest)
    
    #print(independentVariables.head())
    X = independentVariables
  
    # VIF dataframe
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
  
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    return vif_data
                                                 
def checkForMultiCol(df, featuresToTest):
    multicols =[]
    #calculate the Variance Inflation Factor and gives a table with all VIF with the features back
    vifData = calculateVIF(df, featuresToTest)
    #Goes through the table and checks for multicollinearity
    for i in range(0,len(vifData)-1):
            # >= 5 because from 5 multicollinearity is there
            if vifData['VIF'][i] >= 5:
                print(vifData['VIF'][i])
                #saves features that are multicollinearity in list
                multicols +=[vifData['feature'][i]]
    #print(multicols)
    return multicols

def multi_col_test(dataFrame):
    #Test if there is Multicollinearity in the OneHotEncoding
    multicols =[]
    for i in range(0,len(CATEGORICAL_COLUMN_NAMES)):
        #print(list(hotCodedNames[i]))
        multicols +=checkForMultiCol(dataFrame, list(CATEGORICAL_COLUMN_NAMES[i]))
    if (len(multicols)==0):
        return True
    else:
        return False

## 2.2.7 Normalization
Normalize:
- Numerical.
- label encoded data.

Don't normalize:
- Binary Data .
- One Hot Encoded data.

In [14]:
def normalize_data(dataFrame):
    normalizationData = dataFrame[NORMALIZATION_COLUMN_NAMES]

    x = normalizationData.values #returns a numpy array
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    normalizedDf = pd.DataFrame(x_scaled, columns=NORMALIZATION_COLUMN_NAMES)
    #print(sD.columns)
    dataFrame.drop(labels = NORMALIZATION_COLUMN_NAMES, inplace=True, axis = 1)
    #print(sD.columns)

    dataFrame = pd.merge(dataFrame, normalizedDf, left_index=True, right_index=True)
    return dataFrame

## 2.3 -Feature Selection
### 2.3.1 New Feature Split

In [15]:
def get_feature_data(dataFrame, featureColumnNames):    
    featureData = dataFrame.drop(labels = OUTCOME_COLUMN_NAMES, axis = COLUMN_AXIS)
    
    #filter out the columnNames from a specific pool of columnNames: 
    # - surgery features or patient features
    featureData = featureData.filter(featureColumnNames)
    
    # print(featureData)
    return featureData
    
def backward_elimination(X, y):

    #Adding constant column of ones, mandatory for sm.OLS model
    X_1 = sm.add_constant(X)
    
    #Fitting sm.OLS model
    model = sm.OLS(y,X_1).fit()
    #model.pvalues

    #Extract columnsNames
    columnNames = list(X.columns)
    #print(len(columnNames)) #62
    SIGNIFICANCE = 0.05

    #Set the pmax variable to 1. Will be replaced in while loop.
    pmax = 1

    #while the length of the columnNames is not empty (yet).
    #wrapper methods:
    #     Backwards elimination = single features only.
    #     Filter method is = combination of features.
    while (len(columnNames)>0):

        # initialize an empty list for all the pValues.
        p= []

        # make the statistical model and fit it to the data.
        X_1 = X[columnNames]
        X_1['const'] = 1 #TODO FIND OUT WHY THE PROGRAMM SPITS OUT WARNINGS HERE AND FIX IT!
        model = sm.OLS(y,X_1).fit()

        new_arr = delete(model.pvalues.values, (len(model.pvalues.values)-1))

        #make a dataframe with the columnNames and their corresponding P values.
        p = pd.Series(new_arr,index = columnNames)

        # extract the maximum P value of the column.
        pmax = max(p)
        feature_with_p_max = p.idxmax()

        # compare the p value with the significance value.
        # if the pmax is bigger than the significance value, the specific 
        # feature is insignificant and removes it from the list.
        if(pmax>SIGNIFICANCE):
            #print(feature_with_p_max)
            columnNames.remove(feature_with_p_max)

        # if there is no maximum p-value that is insignificant anymore, the while loop will break.
        else:
            break

    selectedFeatures = columnNames

    #print(len(selectedFeatures))
    #print(selectedFeatures)
    
    featuresDataSelected = X.filter(selectedFeatures)
    #print(featuresDataSelected.info())
    
    selectedFeaturesColumnNames = featuresDataSelected.columns
    #print(selectedFeaturesColumnNames)
    
    return featuresDataSelected, list(selectedFeaturesColumnNames)

In [16]:
def get_target_names_string(yColumn):

    #initialize which column will be the y in the model.
    #print(yColumn.unique())

    #initialize the list for the targetnames.
    targetNames = list(yColumn.unique())
    #print(targetNames)

    #initalize for the classification report
    targetNamesString = [str(i) for i in targetNames]
#     print(outcomeData.columns)

    #print(yColumn.unique())
    
    return targetNamesString

### 2.3.2 Inverted Indexes Method

In [17]:
def add_index_to_value_column(temp_dict, column, value, index):
            
    if value not in temp_dict[column]: temp_dict[column][value] = []
    
    #add the index to the specific value of that column
    temp_dict[column][value].extend(index)
    return temp_dict

def create_ii(X_array):
    ii = None

    #create inverted index
    for index, vector in enumerate(X_array):
        #build inverted index and make slots for the columns
        if ii == None: ii = [{} for _ in vector]

        #for every column in this rowVector
        for column, value in enumerate(vector):
            ii = add_index_to_value_column(ii, column, value, [index])
    return ii

def retrieve_vectors_ii(ii, query_vector):
    scores = {}
    
    #for every column in the query vector
    for qcolumn, qvalue in enumerate(query_vector): 
        
        #count the map indexes(documents) containing this word and how many times document appears in ii.
        if qvalue in ii[qcolumn]:
            
            #get the indices that have this column value
            ret_indices = ii[qcolumn][qvalue]
            
            #add the indices to the score for each appearing indices.
            #it counts for every column the reoccuring indices.
            for i in ret_indices:
                if i not in scores:
                    scores[i] = 1
                else: 
                    scores[i] += 1
                    
    # find the highest score: the score for the indices that
    # are most similar. 
    # Retrieve the indices with this score: most similar to the query vector.
    max_val = max(scores.values())
    max_keys = [key for key in scores if scores[key] == max_val]
    return max_keys

def most_frequent(List):
    return max(set(List), key = List.count)

def retrieve_y_values(indices, y_array):
    count = 0
    
    #make a list with the length of the amount of retrieved indices
    y_value = [None]*len(indices)
    
    #count for each appearing index, how many times it appears in total.
    for index in indices:
        y_value[count] = y_array[index]
        count += 1
        
    return y_value


def predict_y_ii (ii, X_array, y_array):
    pred = [None]*len(X_array)
    
    for i, qvector in enumerate(X_array):
        
        #retrieve the similar indices for this query vector
        most_sim_indices = retrieve_vectors_ii(ii, qvector) 
        
        #retrieve the according y values for the most appearing indices.
        y_values = retrieve_y_values(most_sim_indices, y_array)
        
        prediction = most_frequent(y_values)
        pred[i] = prediction
    return pred

## 2.4 Evaluate the models

In [18]:
def reportModelPerformance(model, cv, X, y):
    # evaluate model
    accuracy_scores = model_selection.cross_val_score(model, X, y, scoring='accuracy', cv=cv,n_jobs=-1)
    recall_scores = model_selection.cross_val_score(model, X, y, scoring='recall', cv=cv,n_jobs=-1)
    avPrecicion_scores = model_selection.cross_val_score(model, X, y, scoring='average_precision', cv=cv,n_jobs=-1)
    f1_scores = model_selection.cross_val_score(model, X, y, scoring='f1_weighted', cv=cv,n_jobs=-1)
    f1_scores_macro = model_selection.cross_val_score(model, X, y, scoring='f1_weighted', cv=cv,n_jobs=-1)
    mae = model_selection.cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    rmse = model_selection.cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
   
    # report performance
    print('Accuracy score \t %0.3f with a standard deviation of %0.3f' % (accuracy_scores.mean(), accuracy_scores.std()))
    print("F1 macro score \t %0.3f with a standard deviation of %0.3f" % (f1_scores_macro.mean(), f1_scores_macro.std()))
    print("F1 weighted sc \t %0.3f with a standard deviation of %0.3f" % (f1_scores.mean(), f1_scores.std()))
    
    # NaN because y is an object
#     print("Recall \t{}\t with a standard deviation of {}".format(recall_scores.mean(), recall_scores.std()))
#     print("Average pre \t{}\t with a standard deviation of {}".format(avPrecicion_scores.mean(), avPrecicion_scores.std()))

    # view mean absolute error:
    # the average absolute error between the model prediction and the actual observed data
    # In general, the lower the MAE, the more closely a model is able to predict the actual observations.    
    print ('MAE error \t %0.3f' % (mean(absolute(mae))))
    print ('RMSE error \t %0.3f' % (mean(absolute(rmse))))
    print ()
    

    
def reportPerformance(groundTruth, predictions, report_mode):
    """#reports the quality of the prediction.
    #the value that determines the best classifier, depends on the report_mode
    "continuos" --> RMSE
    "categorical" --> Accuracy
    
    if report_mode == "continuos" the accuracy/F1-Scores is not computed, to avoid problems with regression based classifiers
    """
    if report_mode != "continuos":
        #printing the classification report of the performance
        accuracy=accuracy_score(groundTruth, predictions)
        f1macro=f1_score(groundTruth, predictions, average='macro')
        f1weighted=f1_score(groundTruth, predictions, average='weighted')
        print ('Accuracy score \t %0.3f'%(accuracy))
        print ('F1 Macro score \t %0.3f'%(f1macro))
        print ('F1 Weighted sc \t %0.3f'%(f1weighted))
    
    MAE=mean_absolute_error(groundTruth, predictions)
    RMSE=sqrt(mean_squared_error(groundTruth, predictions))
    
    #     print ('Recall \t{}'.format(recall_score(groundTruth, predictions, average = 'weighted')))
    #     print ('Average prec \t{}'.format(average_precision_score(groundTruth, predictions)))
    print ('MAE error \t %0.3f'%(MAE))
    print ('RMSE error \t %0.3f'%(RMSE))
    print ()
    if report_mode == "continuos":
        return -RMSE
    else:
        return accuracy

# Chapter 3 - The Pipeline

## 3.0 Preparation

#### Constants

In [19]:
FILENAME = 'surgical_case_durations.csv'      # The Filename of the csv file, that we are using.
COLUMN_AXIS = 1                               #
ROW_AXIS = 0                                  #


Here you can choose what you want to predict. Choose between:

'ActualDurationTime' 

'ActualDurationTimeRange' 

In [20]:
# PREDICTED_COLUMN='ActualDurationTime'
PREDICTED_COLUMN=input("Please, Type in your choice 'ActualDurationTime' or 'ActualDurationTimeRange': ")    # The Column that we want to predict using this pipeline. Can be: ActualDurationTimeRange or ActualDurationTime

Please, Type in your choice:ActualDurationTime


#### Lists and Dictionaries

In [21]:
#Dictionary for the column renamings
TO_ENGLISH_COLUMNS={"Geplande operatieduur": "PlannedDurationTime",
                    "Operatieduur": "ActualDurationTime",
                    "Operatietype":"OperationType",
                    "Chirurg":"Surgeon",
                    "Anesthesioloog":"Anesthesiologist",
                    "Benadering":"Approach",
                    "OK":"OperationRoom",
                    "Casustype":"Urgency",
                    "Dagdeel":"PartOfDay",
                    "Leeftijd":"Age",
                    "AF":"AtrialFibrillation",
                    "HLM":"CardiopulmonaryBypassUse",
                    "Geslacht":"Sex",
                    "Aantal anastomosen":"AmountOfBypasses",
                    "Chronische longziekte":"P_ChronicLungDisease",
                    "Extracardiale vaatpathie":"P_extracardialArteriopathy",
                    "Actieve endocarditis": "P_activeEndocarditis",
                    "Hypertensie":"P_Hypertension",
                    "Pulmonale hypertensie":"P_PulmonaleHypertension",
                    "Slechte mobiliteit":"P_PoorMobility",
                    "Hypercholesterolemie":"P_Hypercholesterolemia",
                    "Perifeer vaatlijden":"P_PeripherialVascularDisease",
                    "Linker ventrikel functie":"LeftVentricleFunction",
                    "Nierfunctie":"RenalFunction",
                    "DM":"P_Diabetis",
                    "Eerdere hartchirurgie":"PreviousHeartSurgery",
                    "Kritische preoperatieve status":"CriticalPre-OP",
                    "Myocard infact <90 dagen":"MycordialInfarctionPreSurgery",
                    "Aorta chirurgie":"AorticSurgery",
                    "Ziekenhuis ligduur":"HospitalDays",
                    "IC ligduur":"ICUDays"
                   }

#All features, that are related to the patient
PATIENT_FEATURES= ['Urgency', 'Sex',
       'AtrialFibrillation', 'P_ChronicLungDisease',
       'P_extracardialArteriopathy', 'PreviousHeartSurgery',
       'P_activeEndocarditis', 'CriticalPre-OP',
       'MycordialInfarctionPreSurgery', 'AorticSurgery',
       'P_PulmonaleHypertension', 'LeftVentricleFunction', 'Euroscore1',
       'Euroscore2', 'RenalFunction', 'P_PoorMobility', 'P_Diabetis',
       'P_Hypercholesterolemia', 'P_Hypertension',
       'P_PeripherialVascularDisease', 'CCS', 'NYHA', 'AmountOfBypasses',
       'CardiopulmonaryBypassUse']

#Features that are related to the surgery
SURGERY_FEATURES =[ 'Surgeon', 'Anesthesiologist', 'Approach',
       'OperationRoom','PartOfDay', 'OperationType']
#________________________________________

RANGE_COLUMN_NAMES = ['Age', 'BMI', 'ActualDurationTime',
                         'PlannedDurationTime'] #and overtime

ORDINAL_COLUMN_NAMES = RANGE_COLUMN_NAMES + ['CCS', 'NYHA', 'Urgency',  
                         'LeftVentricleFunction', 'RenalFunction',
                         'P_PulmonaleHypertension']

NON_ORDINAL_COLUMN_NAMES = ['AmountOfBypasses', 'Euroscore1',
                            'Euroscore2', 'HospitalDays', 'ICUDays']

NUMERICAL_COLUMN_NAMES = RANGE_COLUMN_NAMES + NON_ORDINAL_COLUMN_NAMES

CATEGORICAL_COLUMN_NAMES = ['Surgeon', 'OperationRoom', 'OperationType',
                            'Anesthesiologist', 'Approach', 'PartOfDay']
#________________________________________

BINARY_COLUMN_NAMES = ['Sex', 'AtrialFibrillation', 'P_ChronicLungDisease',
                        'P_extracardialArteriopathy', 'PreviousHeartSurgery', 
                        'P_activeEndocarditis', 'CriticalPre-OP',
                        'MycordialInfarctionPreSurgery', 'AorticSurgery',
                        'P_PoorMobility', 'P_Diabetis', 'P_Hypercholesterolemia',
                        'P_Hypertension', 'P_PeripherialVascularDisease', 
                        'CardiopulmonaryBypassUse']

NORMALIZATION_COLUMN_NAMES = ['CCS', 'NYHA', 'Urgency', 'BMIRange',
                       'LeftVentricleFunction', 'RenalFunction',
                       'P_PulmonaleHypertension', 'AgeRange', 'Euroscore1', 
                       'Euroscore2']

OUTCOME_COLUMN_NAMES = ['HospitalDays', 'ICUDays', 'PlannedDurationTime', 
                        'PlannedDurationTimeRange', 'OvertimeRange',
                        'ActualDurationTimeRange','ActualDurationTime']




## 3.1 Importing the Data

In [22]:
# Import the data from the file: 'surgical_case_durations.csv'. Store it in a dataframe
surgicalData = import_df_from_file(FILENAME)

,Operatietype,Chirurg,Anesthesioloog,Benadering,OK,Casustype,Dagdeel,Leeftijd,Geslacht,AF,...,Hypertensie,Perifeer vaatlijden,CCS,NYHA,Aantal anastomosen,HLM,Geplande operatieduur,Operatieduur,Ziekenhuis ligduur,IC ligduur
0,Amputatie teen + Wondtoilet,"6,00",Onbekend,NaN,TOK3,Electief,Middag,51.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.0,52.0,Onbekend,Onbekend
1,Arthroscopische acrominoclaviculaire reconstru...,Ander specialisme,Onbekend,NaN,OK 10,Electief,Middag,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,70.0,Onbekend,Onbekend
2,Ascendensvervanging,"4,00","8,00",Volledige sternotomie,TOK1,Electief,Ochtend,78.0,V,N,...,N,J,0.0,2.0,0.0,N,229.0,170.0,"4,00","2,00"
3,Ascendensvervanging,"7,00","6,00",Volledige sternotomie,TOK2,Electief,Middag,66.0,V,J,...,N,N,2.0,1.0,0.0,N,140.0,190.0,"6,00","1,00"
4,Ascendensvervanging,"6,00",Onbekend,NaN,TOK2,Spoed,Avond,72.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,370.0,480.0,Onbekend,Onbekend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,Wondtoilet,"2,00",Onbekend,NaN,TOK1,Electief,Middag,62.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,78.0,65.0,"44,00","0,00"
4082,Wondtoilet,"4,00",Onbekend,NaN,TOK3,Electief,Middag,62.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.0,25.0,Onbekend,Onbekend
4083,Wondtoilet,"4,00",Onbekend,NaN,TOK3,Spoed < 24 uur,Ochtend,62.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,46.0,39.0,Onbekend,Onbekend
4084,Wondtoilet,"15,00",Onbekend,NaN,TOK2,Electief,Middag,57.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.0,46.0,Onbekend,Onbekend


## 3.2 Preprocessing



### 3.2.0 - Translate column-names to english

In [23]:
# translate the columns to english
surgicalData = surgicalData.rename(columns = TO_ENGLISH_COLUMNS)

### 3.2.1 Add new Columns

#### Overtime Column
Creating the Overtime Column based on the planned- and actual duration time.

In [24]:
# create the overtime column
surgicalData = createOvertimeColumn(surgicalData)
#Handle the OperationType Column


#### OperationType
The OperationType column has a string with all the operations performed on the subject. The operations, that make up a to over 99% of the total operations are included

In [25]:
OperationTypeEncoder=OPTypeEncoder(n=99)
surgicalData=OperationTypeEncoder.fit_transform(surgicalData, 'OperationType')
#surgicalData,OPTypeColumnNames = createOperationTypeColumn(surgicalData, 'OperationType')    

The relevant op types, that cover at least 99% of all operations, are: ['other', 'cabg', 'avr', 'pacemakerdraad tijdelijk', 'mvp', 'mvp shaving', 'wondtoilet', 'tvp', 'mvr']


### 3.2.2 Remove NaNs

In [26]:
#in the dataset we have some "onbekend"-values, that should be NaNs too.
surgicalData = replace_with_nan (surgicalData, value = 'Onbekend')

#drop columns and rows with too much nan
surgicalData, droppedColumns = dropWithTooMuchNan(surgicalData, axis = COLUMN_AXIS)

for removeColumnName in droppedColumns:
    if removeColumnName in RANGE_COLUMN_NAMES:
        RANGE_COLUMN_NAMES.remove(removeColumnName)
    if removeColumnName in ORDINAL_COLUMN_NAMES:
        ORDINAL_COLUMN_NAMES.remove(removeColumnName)
    if removeColumnName in NON_ORDINAL_COLUMN_NAMES:
        NON_ORDINAL_COLUMN_NAMES.remove(removeColumnName)
    if removeColumnName in NUMERICAL_COLUMN_NAMES:
        NUMERICAL_COLUMN_NAMES.remove(removeColumnName)
    if removeColumnName in CATEGORICAL_COLUMN_NAMES:
        CATEGORICAL_COLUMN_NAMES.remove(removeColumnName)
    if removeColumnName in BINARY_COLUMN_NAMES:
        BINARY_COLUMN_NAMES.remove(removeColumnName) 
    if removeColumnName in NORMALIZATION_COLUMN_NAMES:
        NORMALIZATION_COLUMN_NAMES.remove(removeColumnName)
        
surgicalData = dropWithTooMuchNan(surgicalData, axis = ROW_AXIS)


### 3.2.3 Retype the data

In [27]:
# retype Object columns to categorical columns, if possible
surgicalData = convert_df_type(surgicalData, list(surgicalData), 'category')

# retype the columns that are not float64 yet
surgicalData = convert_df_type(surgicalData, NUMERICAL_COLUMN_NAMES, 'float64')

# retype the columns that are not integers yet
#surgicalData = convert_df_type(surgicalData, BINARY_COLUMN_NAMES, 'int8')


['Age', 'BMI', 'ActualDurationTime', 'PlannedDurationTime', 'AmountOfBypasses', 'Euroscore1', 'HospitalDays', 'ICUDays']


### 3.2.4 Outlier removal
Outlier cutting for the categorical outliers: those outliers will be grouped into "other", while the numerical outliers will be removed from the dataset.


In [28]:
#categoricalOutlier Cutting
surgicalData = categorical_outlier_cutting(surgicalData, CATEGORICAL_COLUMN_NAMES, 10)
# display("after: categorical_outlier_cutting", surgicalData)

# #numericalOutlier Cutting
# print(surgicalData.boxplot(column=['ActualDurationTime']))
surgicalData = numerical_outlier_cutting(surgicalData, NUMERICAL_COLUMN_NAMES)
# print(surgicalData.boxplot(column=['ActualDurationTime']))
# display("after: numerical_outlier_cutting", surgicalData)

In [29]:
#Needed for the Duration Generator --> Chapter 4
priorData=surgicalData[:]
surgicalData.head()

,Age,CCS,NYHA,AmountOfBypasses,PlannedDurationTime,ActualDurationTime,Overtime,Surgeon,Anesthesiologist,Approach,...,ICUDays,OPType_other,OPType_cabg,OPType_avr,OPType_pacemakerdraad tijdelijk,OPType_mvp,OPType_mvp shaving,OPType_wondtoilet,OPType_tvp,OPType_mvr
0,66.0,2.0,1.0,0.0,140.0,190.0,50.0,"7,00","6,00",Volledige sternotomie,...,1.0,J,N,N,N,N,N,N,N,N
1,71.0,0.0,1.0,0.0,241.0,239.0,-2.0,"4,00","10,00",Volledige sternotomie,...,1.0,J,N,N,N,N,N,N,N,N
2,66.0,0.0,1.0,0.0,240.0,269.0,29.0,"3,00","15,00",Volledige sternotomie,...,0.0,J,N,N,N,N,N,N,N,N
3,52.0,0.0,1.0,0.0,180.0,305.0,125.0,"1,00","11,00",Volledige sternotomie,...,1.0,J,N,N,N,N,N,N,N,N
4,80.0,2.0,2.0,0.0,218.0,300.0,82.0,"2,00","5,00",Volledige sternotomie,...,4.0,J,N,J,N,N,J,N,N,N


### 3.2.5 Categorical Data 
Creating categorical data from ranged numerical data

In [30]:
surgicalData, column2binNames = to_categorical_range(surgicalData)
          
for removeColumnName in RANGE_COLUMN_NAMES:
    replaceColumnName = removeColumnName+'Range'
    if removeColumnName in RANGE_COLUMN_NAMES:
        RANGE_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), RANGE_COLUMN_NAMES))
    if removeColumnName in ORDINAL_COLUMN_NAMES:
        ORDINAL_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), ORDINAL_COLUMN_NAMES))
    if removeColumnName in NON_ORDINAL_COLUMN_NAMES:
        NON_ORDINAL_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), NON_ORDINAL_COLUMN_NAMES))
    if removeColumnName in NUMERICAL_COLUMN_NAMES:
        NUMERICAL_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), NUMERICAL_COLUMN_NAMES))
    if removeColumnName in CATEGORICAL_COLUMN_NAMES:
        CATEGORICAL_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), CATEGORICAL_COLUMN_NAMES))
    if removeColumnName in BINARY_COLUMN_NAMES:
        BINARY_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), BINARY_COLUMN_NAMES))
    if removeColumnName in NORMALIZATION_COLUMN_NAMES:
        NORMALIZATION_COLUMN_NAMES = list(map(lambda x: x.replace(removeColumnName, replaceColumnName), NORMALIZATION_COLUMN_NAMES))

67-76    694
58-66    417
77-87    315
47-57    244
Name: AgeRange, dtype: int64
[46, 57, 66, 76, 87]
5
['47-57', '58-66', '67-76', '77-87']
4
overweight     819
normal         466
obese          385
underweight      0
Name: BMIRange, dtype: int64
[17.91, 18.5, 24.9, 29.9, 36.1]
5
['underweight', 'normal', 'overweight', 'obese']
4
181-240    665
241-280    443
281-345    305
110-180    192
346-398     65
Name: ActualDurationTimeRange, dtype: int64
[109, 180, 240, 280, 345, 398]
6
['110-180', '181-240', '241-280', '281-345', '346-398']
5
181-240    1043
241-280     408
140-180     119
281-330     100
Name: PlannedDurationTimeRange, dtype: int64
[139, 180, 240, 280, 330]
5
['140-180', '181-240', '241-280', '281-330']
4
In_Time                  633
Overtime_Small           398
Ahead_Of_Time_Medium     312
Overtime_Medium          164
Ahead_Of_Time_Extreme     94
Overtime_Extreme          69
Name: OvertimeRange, dtype: int64
[-165.0, -70, -25, 25, 70, 120, 215.0]
7
['Ahead_Of_Time_Extreme'

### 3.2.6 Encoding

#### Label Encoding
This is done for numerical data that is ordinal.

In [31]:
toBeLE=ORDINAL_COLUMN_NAMES+BINARY_COLUMN_NAMES
surgicalData,columnNameToLE = label_encoding(surgicalData, toBeLE)

1    665
2    443
3    305
0    192
4     65
Name: ActualDurationTimeRange, dtype: int64


#### One Hot Encoding
One hot encoding should be done for the features, that are nominal-scaled.

label encoding is done for categorical data, with string names, so that it can be understood by the ml-algorithms

In [32]:
toBeOHE=CATEGORICAL_COLUMN_NAMES

surgicalData, codedNames,columnNameToOHE  = one_hot_encoding(toBeOHE, surgicalData)

intersectionSFC = set(SURGERY_FEATURES).intersection(CATEGORICAL_COLUMN_NAMES)
intersectionPFC = set(PATIENT_FEATURES).intersection(CATEGORICAL_COLUMN_NAMES)
for columnName in intersectionSFC:
    SURGERY_FEATURES.remove(columnName)
for columnName in intersectionPFC:
    PATIENT_FEATURES.remove(columnName)

#replace the old columns with the encoded ones.
CATEGORICAL_COLUMN_NAMES = list(codedNames)+OperationTypeEncoder.relevant_op_types_Names


#add the encoded columns to the features.
SURGERY_FEATURES += list(codedNames)
PATIENT_FEATURES += list(codedNames)

#TODO: remove intersection categoricalColumnNames 

#from surgery features and from patient features.

# for codedName in codedNames:
#     CATEGORICAL_COLUMN_NAMES.append(list(hotCodedNames[i]))
# for columnName in CATEGORICAL_COLUMN_NAMES:
#     CATEGORICAL_COLUMN_NAMES.remove(columnName)

Surgeon
OperationRoom
Anesthesiologist
Approach
PartOfDay


####  Variance Inflation Factor (VIF)
The VIF values indicate how many times larger the variance would become, to create a dataset that would have been uncorrelated. If this VIF value is smaller than 5, that means no significant multicollinearity appears
within the dataset.
The following code will then produce the output 'True'

In [33]:
print(multi_col_test(surgicalData))
#continue the coding if this is true

True


### 3.2.7 Normalizing

Normalize:
- Numerical.
- label encoded data.

Don't normalize:
- Binary Data .
- One Hot Encoded data.

In [34]:
surgicalData = normalize_data(surgicalData)
#display(surgicalData)

In [35]:
surgicalData.to_csv("PreprocessedSurgicalData.csv", index=False)

# 3.3: Feature Selection

In [36]:
featureData=pd.read_csv("PreprocessedSurgicalData.csv")
featureData.head()

,AmountOfBypasses,PlannedDurationTime,ActualDurationTime,Sex,AtrialFibrillation,P_ChronicLungDisease,P_extracardialArteriopathy,PreviousHeartSurgery,P_activeEndocarditis,CriticalPre-OP,...,PartOfDay_Middag,PartOfDay_Ochtend,PartOfDay_other,CCS,NYHA,Urgency,BMIRange,P_PulmonaleHypertension,AgeRange,Euroscore1
0,0.0,140.0,190.0,1,0,1,0,1,1,1,...,1.0,0.0,0.0,0.5,0.000000,0.0,0.0,1.0,0.333333,0.716137
1,0.0,241.0,239.0,0,1,1,1,1,1,1,...,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.666667,0.575972
2,0.0,240.0,269.0,1,1,1,1,1,1,1,...,0.0,1.0,0.0,0.0,0.000000,0.0,0.5,1.0,0.333333,0.574794
3,0.0,180.0,305.0,1,1,1,1,1,1,1,...,0.0,1.0,0.0,0.0,0.000000,0.0,0.5,1.0,0.000000,0.373969
4,0.0,218.0,300.0,0,1,1,1,1,1,1,...,1.0,0.0,0.0,0.5,0.333333,0.0,0.0,1.0,1.000000,0.365724


In [37]:
#get data that is our X and data that could be our Y
outcomeData = surgicalData[OUTCOME_COLUMN_NAMES]
y = outcomeData[PREDICTED_COLUMN]
#get the targetNamesString for the classification report
targetNamesString = get_target_names_string(y)


#find relevant Features based on the surgery
featureData = get_feature_data(surgicalData, SURGERY_FEATURES)
selectedSurgeryFeaturesData, selectedSurgeryFeatureColumnNames = backward_elimination(featureData, y)
#print(selectedSurgeryFeatureColumnNames)
#find relevant Features based on the patient
featureData = get_feature_data(surgicalData, PATIENT_FEATURES)
selectedPatientFeaturesData, selectedPatientFeatureColumnNames = backward_elimination(featureData, y)
#print(selectedPatientFeatureColumnNames)

selectedFeatureColumnNames=selectedSurgeryFeatureColumnNames+selectedPatientFeatureColumnNames
selectedFeatureNames=list(selectedFeatureColumnNames)
print("The selected Features are:",selectedFeatureNames)

featureData= surgicalData[selectedFeatureNames.copy()]

The selected Features are: ['OPType_other', 'OPType_cabg', 'OPType_avr', 'OPType_pacemakerdraad tijdelijk', 'OPType_mvp', 'OPType_mvp shaving', 'OPType_wondtoilet', 'OPType_tvp', 'OPType_mvr', 'AtrialFibrillation', 'PreviousHeartSurgery', 'P_activeEndocarditis', 'AorticSurgery', 'P_PulmonaleHypertension', 'P_Hypertension', 'CCS', 'NYHA', 'AmountOfBypasses', 'CardiopulmonaryBypassUse']


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
<ipython-input-15-52db3c35653c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_1['const'] = 1 #TODO FIND OUT WHY THE PROGRAMM SPITS OUT WARNINGS HERE AND FIX IT!
<ipython-input-15-52db3c35653c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [38]:
featureData.to_csv("SelectedFeatureData.csv", index=False)

# Chapter 3.4: Evaluation

In [39]:
featureData=pd.read_csv("SelectedFeatureData.csv")
featureData.head()

,OPType_other,OPType_cabg,OPType_avr,OPType_pacemakerdraad tijdelijk,OPType_mvp,OPType_mvp shaving,OPType_wondtoilet,OPType_tvp,OPType_mvr,AtrialFibrillation,PreviousHeartSurgery,P_activeEndocarditis,AorticSurgery,P_PulmonaleHypertension,P_Hypertension,CCS,NYHA,AmountOfBypasses,CardiopulmonaryBypassUse
0,0,1,1,1,1,1,0,1,1,0,1,1,0,1.0,1,0.5,0.000000,0.0,1
1,0,1,1,1,1,1,0,1,1,1,1,1,0,1.0,0,0.0,0.000000,0.0,1
2,0,1,1,1,1,1,0,1,1,1,1,1,0,1.0,0,0.0,0.000000,0.0,0
3,0,1,1,1,1,1,0,1,1,1,1,1,0,1.0,1,0.0,0.000000,0.0,0
4,0,1,0,1,1,0,0,1,1,1,1,1,1,1.0,1,0.5,0.333333,0.0,1


## 3.4.1 train-test split

In [40]:
X=featureData
#As we want to get the performance metrics for the original prediction as well and need the
#same split for this, we add 'PlannedDurationTime' here before the train-test-split
X = pd.merge(X, surgicalData['PlannedDurationTime'], left_index=True, right_index=True)
X = pd.merge(X, surgicalData['PlannedDurationTimeRange'], left_index=True, right_index=True)
#split the dataset into training (70%) and testing (30%) sets
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,test_size=0.3,random_state=randomSeed)


#Now that the split is done we split the 'Planned Duration Time' from feature split
y_test_Original = pd.merge(X_test['PlannedDurationTime'][:],X_test['PlannedDurationTimeRange'][:], left_index=True, right_index=True)
X_train.drop(labels=['PlannedDurationTime','PlannedDurationTimeRange'],axis=COLUMN_AXIS, inplace= True)
X_test.drop(labels=['PlannedDurationTime','PlannedDurationTimeRange'],axis=COLUMN_AXIS, inplace= True)

## 3.4.2 Choose a classifier

### Dummy-Classifier to get a Baseline

In [41]:
from sklearn.dummy import DummyClassifier
dummy_clfs=[DummyClassifier(strategy="most_frequent"),
            DummyClassifier(strategy="prior"),
            DummyClassifier(strategy="stratified"),
            DummyClassifier(strategy="uniform")
           ]

### Regression Based Classifier
Linear Regression, LogisticRegression, Support Vector Machine

In [42]:
from sklearn.dummy import DummyRegressor
reg_clfs=[DummyRegressor(strategy="mean"),
          LinearRegression(),
          svm.SVC(random_state=randomSeed)]

### Tree-Based Classifiers

In [43]:
from sklearn.ensemble import RandomForestClassifier

tree_clfs=[tree.DecisionTreeClassifier(max_depth = 1, random_state=randomSeed),
           RandomForestClassifier(n_estimators=100,max_depth=5, random_state=randomSeed)
          ]

### Clustering-Based Methods

In [44]:
1# affinity propagation clustering
from sklearn.cluster import AffinityPropagation, KMeans, MeanShift, OPTICS, SpectralClustering, MiniBatchKMeans, Birch
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier

cluster_clfs=[
    
    AffinityPropagation(damping=0.9, random_state=randomSeed),
    Birch(threshold=0.01, n_clusters=2),
    KMeans(n_clusters=2, random_state=randomSeed),
    MiniBatchKMeans(n_clusters=2, random_state=randomSeed),
    #MeanShift(),
    #OPTICS(eps=0.8, min_samples=10, random_state=randomSeed),
    #SpectralClustering(n_clusters=2, random_state=randomSeed),
    GaussianMixture(n_components=2, random_state=randomSeed),
    KNeighborsClassifier()
]

## 3.4.3 Evaluate the models

In [45]:

bestClassifier=""

if PREDICTED_COLUMN=="ActualDurationTime":
    report_mode="continuos"
    bestEvalScore=-100000
    planned_y_test = y_test_Original['PlannedDurationTime']
if PREDICTED_COLUMN=="ActualDurationTimeRange":
    report_mode="categorical"
    bestEvalScore=0
    planned_y_test = y_test_Original['PlannedDurationTimeRange']

    
print("dummy-classifier\n")    
strategy=["most_frequent","prior","stratified","uniform"]
for c,clf in enumerate(dummy_clfs):
    print(clf.__class__.__name__+"-Strategy:"+strategy[c])
    clf.fit(X_train, y_train)
    predicted=clf.predict(X_test)
    EvalScore=reportPerformance(y_test, predicted,report_mode)
    if EvalScore>bestEvalScore:
        bestEvalScore=EvalScore
        bestClassifier=clf    

#only useful if we look at continuos data 
if PREDICTED_COLUMN=="ActualDurationTime":
    print("regression based\n")
    for clf in reg_clfs:
        print(clf.__class__.__name__)
        clf.fit(X_train, y_train)
        predicted=clf.predict(X_test)
        EvalScore=reportPerformance(y_test, predicted, report_mode)
        if EvalScore>bestEvalScore:
            bestEvalScore=EvalScore
            bestClassifier=clf

print("tree-based\n")    
for clf in tree_clfs:
    print(clf.__class__.__name__)
    clf.fit(X_train, y_train)
    predicted=clf.predict(X_test)
    EvalScore=reportPerformance(y_test, predicted,report_mode)
    if EvalScore>bestEvalScore:
        bestEvalScore=EvalScore
        bestClassifier=clf
        
print("clustering based\n")    
for clf in cluster_clfs:
    print(clf.__class__.__name__)
    clf.fit(X_train, y_train)
    predicted=clf.predict(X_test)
    EvalScore=reportPerformance(y_test, predicted,report_mode)
    if EvalScore>bestEvalScore:
        bestEvalScore=EvalScore
        bestClassifier=clf

print("InverseIndexes")
ii = create_ii(X_train.to_numpy())
predicted = predict_y_ii(ii, X_test.to_numpy(), y_train.to_numpy())
EvalScore=reportPerformance(y_test, predicted,report_mode)
if EvalScore>bestEvalScore:
    bestEvalScore=EvalScore
    bestClassifier=clf
    
print("Original Prediction")

predicted = planned_y_test
EvalScore=reportPerformance(y_test, predicted,report_mode)
if EvalScore>bestEvalScore:
    bestEvalScore=EvalScore
    bestClassifier=clf
        
if report_mode=="continuos":
    print("The classifier with the lowest RMSE (",str(-bestEvalScore),") is", bestClassifier.__class__.__name__)
if report_mode=="categorical":
    print("The classifier with the highest acccuracy (",bestEvalScore,") is", bestClassifier.__class__.__name__)

dummy-classifier

DummyClassifier-Strategy:most_frequent
MAE error 	 44.950
RMSE error 	 57.003

DummyClassifier-Strategy:prior
MAE error 	 44.950
RMSE error 	 57.003

DummyClassifier-Strategy:stratified
MAE error 	 59.507
RMSE error 	 76.560

DummyClassifier-Strategy:uniform
MAE error 	 77.828
RMSE error 	 93.522

regression based

DummyRegressor
MAE error 	 43.616
RMSE error 	 54.959

LinearRegression
MAE error 	 35.918
RMSE error 	 45.910

SVC
MAE error 	 42.513
RMSE error 	 54.535

tree-based

DecisionTreeClassifier
MAE error 	 41.186
RMSE error 	 51.907

RandomForestClassifier
MAE error 	 40.062
RMSE error 	 50.970

clustering based

AffinityPropagation


C:\Users\xlbok\AppData\Roaming\Python\Python38\site-packages\sklearn\cluster\_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(
C:\Users\xlbok\AppData\Roaming\Python\Python38\site-packages\sklearn\cluster\_affinity_propagation.py:528: ConvergenceWarning: This model does not have any cluster centers because affinity propagation did not converge. Labeling every sample as '-1'.
  warnings.warn(
C:\Users\xlbok\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but Birch was fitted with feature names
  warnings.warn(


MAE error 	 246.908
RMSE error 	 252.903

Birch
MAE error 	 245.575
RMSE error 	 251.615

KMeans
MAE error 	 245.521
RMSE error 	 251.567

MiniBatchKMeans
MAE error 	 245.521
RMSE error 	 251.567

GaussianMixture
MAE error 	 245.559
RMSE error 	 251.595

KNeighborsClassifier
MAE error 	 61.583
RMSE error 	 76.545

InverseIndexes


C:\Users\xlbok\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


MAE error 	 45.752
RMSE error 	 58.877

Original Prediction
MAE error 	 45.401
RMSE error 	 59.082

The classifier with the lowest RMSE ( 45.91019239849399 ) is LinearRegression


# __________________

# Chapter 4: Duration Time Generator
- Duration Time Generator

In [46]:
#for hot encoding
hotColumnNames = ['Surgeon', 'OperationRoom', 'Anesthesiologist', 'Approach', 
                  'PartOfDay']  #TODO Ähnlich zu CATEGORICAL_COLUMN_NAMES

#for label encoding
catOrdinalColumnNames = ['CCS', 'NYHA', 'Urgency','BMIRange',
                       'LeftVentricleFunction', 'RenalFunction',
                       'P_PulmonaleHypertension', 'AgeRange','OvertimeRange','ActualTimeRange']

#for binary labeling
binaryColumnNames=['Sex', 'AtrialFibrillation', 'P_ChronicLungDisease',
                 'P_extracardialArteriopathy', 'PreviousHeartSurgery', 
                 'P_activeEndocarditis', 'CriticalPre-OP',
                 'MycordialInfarctionPreSurgery', 'AorticSurgery',
                 'P_PoorMobility', 'P_Diabetis', 'P_Hypercholesterolemia',
                 'P_Hypertension', 'P_PeripherialVascularDisease', 'CardiopulmonaryBypassUse']

 ## 4.1 Varibales Needed:
- please fill the surgery's Data in

Information about the code below:
It consists of multiple lists of characteristics. These lists are seperated into three types:

- Patient Data: All the information about the patient
- Surgery Data: All the information about the surgery
- Outcome Data: All the information about the outcome of the surgery. This is asked after the operation, to retrain the model, when enough people have commited the data to us.


In [47]:
#Entries that consider data that is clear after the Surgery
outcomeData =['ActualDurationTime',
              'ICUDays',
              'HospitalDays']

#Prior_Entries, e.g. the data, that is entered before the operation #only ask for necessary data 
priorEntries= list(set(SURGERY_FEATURES + PATIENT_FEATURES)&set(priorData.columns))


Out of the above we define if we want to use categorical or numerical questioning for our data.

- categoryPriorEntries -> Categorical Question
- numericalPriorEntries -> numerical Question


In [48]:
#choice category entries, e.g. entries that are prior entries and categorical
categoryPriorEntries = list(set(CATEGORICAL_COLUMN_NAMES+catOrdinalColumnNames+binaryColumnNames)&set(priorEntries))

#numerical Entries, e.g. entries that are prior entries and numerical
numericalPriorEntries = [Entry for Entry in priorEntries if Entry not in categoryPriorEntries]

Lets see, how many entries we have per category:

In [49]:
print('Len prior Entries', len(priorEntries))
print('Len cat Entries', len(categoryPriorEntries))
print('Len num Entries', len(numericalPriorEntries))

Len prior Entries 35
Len cat Entries 33
Len num Entries 2


## 4.2 Collect needed Data

In [50]:
#generate a dictionary, that maps from the column-name to the answering options
categoryPriorEntries=list(set(priorData.columns)&set(categoryPriorEntries))
optionsForEntry={}
for c,entry in enumerate(categoryPriorEntries):
    data=set(str(i) for i in priorData[entry].unique())
    optionsForEntry[entry] = data

In [51]:
def askUserForCategory(name: str, options: set) ->str:
    """
    asks the user for the correct entry for their patient.
    
    Input:
        name: the name of the entry which is currently in question.
        options: is a set with all the options for the entry.
        
    
    returns:
        selected: is a string with the choosen option
    """
    #sort the options alphabetically or numerical
    options= sorted(options[name])
    
    print('Please fill in the number that represents your data for',name,':')
    
    #print all the options with a representative
    for index,optionName in enumerate(options):
        print(str(index+1) + ')\t' + optionName)
    
    #check for valid input
    lenListOptions = len(options)
    while True:
        inputRaw = input(name + ': ')
        try:
            inputNo = int(inputRaw) - 1
            if inputNo > -1 and inputNo < lenListOptions:
                selected = list(options)[inputNo]
                print('Selected ' +  name + ': ' + selected)
                return selected
            else:
                print('Please select a valid ' + name + ' number')
        except ValueError: 
            print('Please fill in the index of your choice, not the name.')

The following code will ask the user for the characteristic that is relevant for the entry

In [52]:
def get_new_patient_data():
    # create a dict, that saves all the Entries prior to the operation
    dictDataEntries= dict.fromkeys(priorEntries) 
    
    # Go through all catgeorical Data points and ask for entry
    for i in range(0, len(categoryPriorEntries)):
        print()
        entry = categoryPriorEntries[i]
        newDataPoint = askUserForCategory(entry,optionsForEntry)
        dictDataEntries[entry]= [newDataPoint]
        #categoryPriorEntries[i][1]=  newDataPoint
        
    # Go through all numerical Data points and ask for entry
    
    for i in range(0, len(numericalPriorEntries)):
        entry = numericalPriorEntries[i]
        print()
        newDataPoint =input('Fill in the numerical value for '+ entry + ': ')
        dictDataEntries[entry]= [newDataPoint]

    newDataDF=pd.DataFrame.from_dict(dictDataEntries)
    
    return newDataDF



In [ ]:
# newDataDF=get_new_patient_data()
newDataDF.to_csv("new_data.csv", index= False)

# 5.3 Transform Data According to Preprocessing

In [ ]:
newDataDF=pd.read_csv("new_data.csv")

In [ ]:
newDataDF.head()

#### Encode Labels /One hot encode

In [ ]:
for columnName in newDataDF.columns:
    
    if columnName in columnNameToLE.keys():
        print("LE",columnName)
        newDataDF[columnName] = columnNameToLE[columnName].transform(newDataDF[columnName])
    elif columnName in columnNameToOHE.keys():
        print("OHE",columnName)
        #create OHE
        enc=columnNameToOHE[columnName]
        data=newDataDF[columnName]
        hotCodedArray=enc.transform([list(data)]).toarray()
        #Find out columnames
        hotCodedcolumns=[]
        for category in list(enc.categories_[0]):
            hotCodedcolumns+=[columnName+"_"+str(category)]
        
        #create a new df, using the hot coded columns as Column Names
        hotCoded = pd.DataFrame(hotCodedArray, columns = hotCodedcolumns)

        #remove old data column and add new dataframe
        newDataDF.drop(labels=columnName,axis=COLUMN_AXIS, inplace= True)
        newDataDF = pd.merge(newDataDF, hotCoded, left_index=True, right_index=True)

In [ ]:
newDataDF.head()

remove features, that have not been selected

In [ ]:

print("The selected Features are:",selectedFeatureNames)

featureData= surgicalData[selectedFeatureNames].copy()

## 5.4 Predict new Data


In [ ]:
# load best model
patient_data=featureData
prediction=bestClassifier.predict(featureData)

#check if it has a bin or not
if PREDICTED_COLUMN in column2binNames.keys():
    print("The assumed value for",PREDICTED_COLUMN,"in this operation is", column2binNames[PREDICTED_COLUMN][prediction[0]])
else:
    print("The assumed value for",PREDICTED_COLUMN,"in this operation is", prediction[0])
#predict

# 